In [1]:
pip install networkx

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
import folium
import networkx as nx
import random
import math

df = pd.read_csv('D:/ITS/Study/Semester 3/A.I Concepts/Final Project/archive/open_pubs.csv')

#Cleaning
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.dropna(subset=['latitude', 'longitude'])
df = df.drop_duplicates(subset=['name'], keep='last')

# Choose a city to limit the dataset size
chosen_city = "City of London"
df_chosen_city = df[df['local_authority'] == chosen_city].head(10)  # limit it to 5 pubs for now

# OSRM function to get the shortest route and distance between two locations
def get_osrm_route(start_coords, end_coords):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    
    if response.status_code == 200:
        route_data = response.json()
        distance_meters = route_data['routes'][0]['distance']  # Distance in meters
        route_coords = route_data['routes'][0]['geometry']['coordinates']
        return route_coords, distance_meters
    else:
        return None, None

# Initialize a Folium map
m = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

# Get pub locations and names
pub_locations = list(zip(df_chosen_city['latitude'], df_chosen_city['longitude']))  # Condense coordinates into tuples
pub_names = df_chosen_city['name'].tolist()  # List of pub names

# Add pub markers to the map
for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}",).add_to(m)

# Initialize an empty graph using networkx
G = nx.Graph()

# Add nodes (pubs) to the graph
for i, pub_name in enumerate(pub_names):
    G.add_node(pub_name, pos=pub_locations[i])

# Add edges (routes between pubs) and calculate distances
total_distance = 0

for i in range(len(pub_locations) - 1):
    for j in range(i + 1, len(pub_locations)):  # Connect all pubs (fully connected graph)
        start_coords = pub_locations[i]
        end_coords = pub_locations[j]
        
        # Get route coordinates and distance from OSRM
        route, distance_meters = get_osrm_route(start_coords, end_coords)
        
        if route:
            # Convert distance to kilometers
            distance_km = distance_meters / 1000
            total_distance += distance_km
            
            # Convert route coordinates to lat-lng for folium display
            route_latlong = [(coord[1], coord[0]) for coord in route]
            
            # Add route as polyline to the map
            folium.PolyLine(locations=route_latlong, color='blue', weight=2.5, opacity=0.8).add_to(m)
            
            # Add an edge to the graph with distance as the weight
            G.add_edge(pub_names[i], pub_names[j], weight=distance_km)
            


# Display the map
m


KeyboardInterrupt: 

In [4]:
import time 

In [11]:

def calculate_route_distance(route, graph):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += graph[route[i]][route[i + 1]]['weight']
    return total_distance

def simulated_annealing(graph, initial_route):
    current_route = initial_route.copy()
    current_distance = calculate_route_distance(current_route, graph)
    best_route = current_route
    best_distance = current_distance

    temperature = 1000.0
    cooling_rate = 0.9999
    while temperature > 1:
        new_route = current_route.copy()
        i, j = random.sample(range(len(new_route)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        
        new_distance = calculate_route_distance(new_route, graph)
        if new_distance < current_distance or random.random() < math.exp((current_distance - new_distance) / temperature):
            current_route = new_route
            current_distance = new_distance
            
            if current_distance < best_distance:
                best_route = current_route
                best_distance = current_distance

        temperature *= cooling_rate

    return best_route, best_distance

start_time = time.time()


initial_route = pub_names

optimized_route, optimized_distance = simulated_annealing(G, initial_route)

optimized_route.append(optimized_route[0]) 

end_time = time.time()
elapsed_time = end_time - start_time

print("Optimized Route:", " -> ".join(optimized_route))
print(f"Optimized Distance: {optimized_distance:.2f} km")
print(f"Simulated Annealing Time: {elapsed_time:.4f} seconds")

m_optimized = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}").add_to(m_optimized)

for i in range(len(optimized_route) - 1):
    start_pub = optimized_route[i]
    end_pub = optimized_route[i + 1]
    
    start_coords = G.nodes[start_pub]['pos']
    end_coords = G.nodes[end_pub]['pos']
    
    route, _ = get_osrm_route(start_coords, end_coords)
    if route:
        route_latlong = [(coord[1], coord[0]) for coord in route]
        
        folium.PolyLine(locations=route_latlong, color='red', weight=2.5, opacity=0.8).add_to(m_optimized)

# Display the optimized map
m_optimized


Optimized Route: El Vino Co Ltd -> Corney & Barrow, Unit 3, Stock Exch -> Bottlescrue -> Balls Brothers Wine Bar -> Pause -> Crosse Keys PH -> One Under Lime -> Emperor Wine Bar -> Shaws Booksellers -> Jamies St Mary At Hill -> Davys, 1 Plantation Place -> Punch Tavern -> Corney & Barrow Wine Bar -> Sky Bar, Grange St Pauls Hotel -> Goodman City Ltd -> City Golf & Health Clubs -> Quarter Jacks, Grange St Pauls Hotel -> Searcy`s Artiste`s Bar, Level - 1 -> Balls Brothers Austin Friars -> City Boot -> George Bar -> Forge -> Merchant House -> Planet of the Grapes Ltd -> Abbey -> Planet Of The Grapes -> City Sports Pub and Grill -> Circle Bar, Level 0 -> Enoteca Rabezzana -> Karaoke Box -> Balls Brothers Ltd -> Harry's Bar & Pizzeria -> Searcys Bars @ GSM & D -> Oriole -> Fox Fine Wines -> Forge Bar & Club -> Harrild and Sons -> City of London Distillery -> Rudd's -> Devonshire Terrace -> Liberty Bounds -> Jamaica Wine House -> Bedales Wine Bar -> East India Arms -> Sir John Hawkshaw -> Ja

In [1]:
from concurrent.futures import ThreadPoolExecutor

In [2]:
##genetic algorithm

# Fitness function to calculate the total distance of a route
def calculate_route_distance(route, graph):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += graph[route[i]][route[i + 1]]['weight']
    total_distance += graph[route[-1]][route[0]]['weight']  # Complete the cycle
    return total_distance

# Create an initial population of random routes
def create_initial_population(population_size, pub_names):
    population = []
    for _ in range(population_size):
        route = pub_names[:]
        random.shuffle(route)
        population.append(route)
    return population

# Select the best individuals for mating (tournament selection)
def selection(population, graph, tournament_size=5):
    selected = random.sample(population, tournament_size)
    selected.sort(key=lambda x: calculate_route_distance(x, graph))
    return selected[0]  # Return the best individual in the tournament

# Crossover function to create a child route
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    # Inherit a subset from parent1
    child[start:end] = parent1[start:end]
    # Fill remaining positions with genes from parent2 in order
    ptr = 0
    for gene in parent2:
        if gene not in child:
            while child[ptr] is not None:
                ptr += 1
            child[ptr] = gene
    return child

# Mutation function to introduce some variations
def mutate(route, mutation_rate=0.01):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]  # Swap two pubs
    return route

# Parallelized fitness evaluation using ThreadPoolExecutor
def calculate_fitness_parallel(population, graph):
    with ThreadPoolExecutor() as executor:
        distances = list(executor.map(lambda route: calculate_route_distance(route, graph), population))
    return distances

# Main Genetic Algorithm function
def genetic_algorithm(graph, pub_names, population_size=100, generations=500, mutation_rate=0.01):
    population = create_initial_population(population_size, pub_names)
    best_route = min(population, key=lambda x: calculate_route_distance(x, graph))
    best_distance = calculate_route_distance(best_route, graph)

    for generation in range(generations):
        new_population = []

        # Create new population through selection, crossover, and mutation
        for _ in range(population_size):
            parent1 = selection(population, graph)
            parent2 = selection(population, graph)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        # Update population and track the best route
        distances = calculate_fitness_parallel(new_population, graph)
        new_population_sorted = [x for _, x in sorted(zip(distances, new_population), key=lambda x: x[0])]

        population = new_population_sorted
        current_best_route = new_population_sorted[0]
        current_best_distance = distances[0]

        if current_best_distance < best_distance:
            best_route = current_best_route
            best_distance = current_best_distance

        # Optionally, print progress every 50 generations
        if generation % 50 == 0:
            print(f"Generation {generation}: Best Distance = {best_distance:.2f} km")

    return best_route, best_distance

# Running the Genetic Algorithm
start_time = time.time()

# Initial route setup and running the algorithm
optimized_route, optimized_distance = genetic_algorithm(G, pub_names, population_size=100, generations=500, mutation_rate=0.01)

end_time = time.time()
elapsed_time = end_time - start_time

print("Optimized Route:", " -> ".join(optimized_route))
print(f"Optimized Distance: {optimized_distance:.2f} km")
print(f"Genetic Algorithm Time: {elapsed_time:.4f} seconds")

# Visualizing the optimized route on a map
m_optimized_genetic = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

# Add markers and optimized route to the map
for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}").add_to(m_optimized_genetic)

# Plot optimized route on the map
optimized_route.append(optimized_route[0])  # Complete the cycle
for i in range(len(optimized_route) - 1):
    start_pub = optimized_route[i]
    end_pub = optimized_route[i + 1]
    
    start_coords = G.nodes[start_pub]['pos']
    end_coords = G.nodes[end_pub]['pos']
    
    route, _ = get_osrm_route(start_coords, end_coords)
    if route:
        route_latlong = [(coord[1], coord[0]) for coord in route]
        
        folium.PolyLine(locations=route_latlong, color='green', weight=2.5, opacity=0.8).add_to(m_optimized_genetic)

# Display the optimized map
m_optimized_genetic




NameError: name 'time' is not defined

In [16]:
m_optimized_genetic.save("genetic_alg.html")